In [ ]:
#hide
from nbdev import *
%nbdev_default_export release

Cells will be exported to nbdev.release,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Create release notes

> Release notes auto-generated from GitHub issues

In [ ]:
#export
from datetime import datetime
from textwrap import fill
from urllib.request import Request,urlopen
from urllib.error import HTTPError
from urllib.parse import urlencode
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from configparser import ConfigParser
import os,shutil,json,subprocess

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
#export
GH_HOST = "https://api.github.com"

In [ ]:
#export
def _issue_txt(issue):
    res = f'- {issue["title"].strip()} ([#{issue["number"]}]({issue["url"]}))\n'
    body = issue['body']
    if not body: return res
    return res + fill(body.strip(), initial_indent="  - ", subsequent_indent="    ") + "\n"

def _issues_txt(iss, label):
    if not iss: return ''
    res = f"### {label}\n\n"
    return res + '\n'.join(map(_issue_txt, iss))

def _config(cfg_name="settings.ini"):
    cfg_path = Path().absolute()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    config_file = cfg_path/cfg_name
    assert config_file.exists(), f"Couldn't find {cfg_name}"
    config = ConfigParser()
    config.read(config_file)
    return config['DEFAULT'],cfg_path

## Helper functions

In [ ]:
#export
def run_proc(*args):
    res = subprocess.run(args, capture_output=True)
    if res.returncode: raise IOError(subprocess.stdout + ";;" + subprocess.stderr)
    return res.stdout

In [ ]:
#export
def do_request(url, post=False, headers=None, **data):
    "Call GET or json-encoded POST on `url`, depending on `post`"
    if data:
        if post: data = json.dumps(data).encode('ascii')
        else:
            url += "?" + urlencode(data)
            data = None
    with urlopen(Request(url, headers=headers, data=data or None)) as res: return json.loads(res.read())

## FastRelease -

In [ ]:
#export
class FastRelease:
    def __init__(self, owner=None, repo=None, token=None, **groups):
        "Create CHANGELOG.md from GitHub issues"
        if not groups: groups = dict(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")
        self.cfg,cfg_path = _config()
        os.chdir(cfg_path)
        if not owner: owner = self.cfg['user']
        if not repo:  repo  = self.cfg['lib_name']
        if not token:
            assert Path('token').exists, "Failed to find token"
            token = Path('token').read_text().strip()
        self.owner,self.repo,self.token,self.groups = owner,repo,token,groups
        self.headers = { 'Authorization' : f'token {token}' }
        self.repo_url = f"{GH_HOST}/repos/{owner}/{repo}"

    def gh(self, path, complete=False, post=False, **data):
        "Call GitHub API `path`"
        if not complete: path = f"{self.repo_url}/{path}"
        return do_request(path, headers=self.headers, post=post, **data)

    def _tag_date(self, tag):
        try: tag_d = self.gh(f"git/ref/tags/{tag}")
        except HTTPError: raise Exception(f"Failed to find tag {tag}")
        commit_d = self.gh(tag_d["object"]["url"], complete=True)
        self.commit_date = commit_d["committer"]["date"]
        return self.commit_date

    def _issues(self, label):
        return self.gh("issues", state='closed', sort='created', filter='all',
                       since=self.commit_date, labels=label)
    
    def _issue_groups(self):
        with ProcessPoolExecutor() as ex: return ex.map(self._issues, self.groups.keys())
        
    def latest_release(self):
        "Tag for the latest release"
        return self.gh("releases/latest")["tag_name"]

    def changelog(self, debug=False):
        "Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`"
        fn = 'CHANGELOG.md'
        txt = Path(fn).read_text()
        marker = '<!-- do not remove -->\n'
        latest = self.latest_release()
        self._tag_date(latest)
        res = f"\n## {self.cfg['version']}\n"
        issues = self._issue_groups()
        res += '\n'.join(_issues_txt(*o) for o in zip(issues, self.groups.values()))
        res = txt.replace(marker, marker+res+"\n")
        if debug: return res
        shutil.copy(fn, fn+".bak")
        Path(fn).write_text(res)

    def release(self):
        ver = self.cfg['version']
        run_proc('git', 'tag', ver)
        run_proc('git', 'push', '--tags')
        run_proc('git', 'pull', '--tags')
#         data = json.dumps({"tag_name": ver, "name": ver}).encode('ascii')
        rel.gh("releases", post=True, tag_name=ver, name=ver)

In [ ]:
rel = FastRelease(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")

In [ ]:
rel.release()

In [ ]:
l = rel.latest_release()
l

'1.0.10'

To create a markdown changelog, first create a `FastRelease` object, passing a mapping from GitHub labels to markdown titles. Put your github token in a file named `token` at the root of your repo.

In [ ]:
show_doc(FastRelease.changelog)

<h4 id="FastRelease.changelog" class="doc_header"><code>FastRelease.changelog</code><a href="__main__.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>FastRelease.changelog</code>(**`debug`**=*`False`*)

Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`

In [ ]:
# rel = FastRelease(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")
# print(rel.changelog(debug=True))

### Utility methods

In [ ]:
show_doc(FastRelease.latest_release)

<h4 id="FastRelease.latest_release" class="doc_header"><code>FastRelease.latest_release</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>FastRelease.latest_release</code>()

Tag for the latest release

In [ ]:
show_doc(FastRelease.gh)

<h4 id="FastRelease.gh" class="doc_header"><code>FastRelease.gh</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>FastRelease.gh</code>(**`path`**, **`complete`**=*`False`*)

Call GitHub API `path`

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 05a_conda.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 10_release.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted magic_flags.ipynb.
Converted nbdev_callbacks.ipynb.
Converted tutorial.ipynb.
